In [1]:
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr
session = WolframLanguageSession()

In [2]:
session.evaluate(wlexpr('Range[5]'))

PackedArray([1, 2, 3, 4, 5], dtype=int8)

In [4]:
session.terminate()